### Name   :- Maddula Jagadhesh
### Reg No:- 12018350
### Roll No:- RK20UPA13
### section:- K20UP

### ONLINE RETAIL DATASET ANALYSIS

### Information about the Dataset

1. This is a transactional data set which contains all the transactions occurring between 01/12/2010 and 09/12/2011 for a UK-based and registered non-store online retail.
2. The company mainly sells unique all-occasion gifts. Many customers of the company are wholesalers.
3. This dataset contains 541909 observations/records with over 8 different attributes.


#### Features -


InvoiceNo<br>
StockCode<br>
Description<br>
Quantity<br>
Invoice Date<br>
UnitPrice<br>
CustomerID<br>
Country<br>

### Questions -

1. Based on your understanding of the data, what kind of business is this company in?
2. Analyze the sales performance of this company, and provide your insights regarding the same
3. Based on your analysis of the data, what are potential areas of improvement for the business?
4. What are additional business problems that can be analyzed using this data. Support with explanation.
5.  Most of the customers belongs to which Country?
6. To find the total amount of sales for each country and see which country has the highest sales.
7. Which month of the year have more sales and reason behind it.
8. Which countries have poor sales and what can be the factors affecting the sales and how we can improve them.
9.  Make a good conclusion for the given dataset from the insights we got after each analysis.


The Answers are provided below with proper analysis.

### Importing required libraries

In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib import pyplot
import missingno as msno
import warnings
warnings.filterwarnings('ignore')
sns.set(style="darkgrid")
%matplotlib inline
import plotly
import plotly.express as px

### 1.Reading dataset into Notebook

In [ ]:
#using pandas we read the excel dataset
df = pd.read_excel("Online Retail.xlsx")

### 2.Basic exploration and data preprocessing

In [ ]:
#2.Basic-exploration-and-data-preprocessing##Let's have a look how does the data looks like:
df.head(10)

#### Number of rows and columns in data set

In [ ]:
print("Number of rows in dataset is: ", df.shape[0])
print("Number of columns in dataset is: ", df.shape[1])

#### Finding the datatypes of variables

In [ ]:
df.dtypes

df.info(). The info() method prints information about the DataFrame. The information contains the number of columns, column labels, column data types, memory usage, range index, and the number of cells in each column (non-null values). Note: the info() method actually prints the info.

In [ ]:
df.info()

Here we will get some information about datatypes of the columns and number of non zero rows in each column. From the information above we can conclude that column Description and CustomerID columns contains some missing values. This fact will be examined in further steps.

#### finding the number of null values present in the dataset

In [ ]:
df.isnull().sum()

Here you can see there are 1454 null values in Description, and there are 135080 null values in coustmer ID column as well  which we can remove further steps.

I have used Missingno library. It is simple to use Python library that provides a series of visualisations to understand the presence and distribution of missing data within a pandas dataframe. This can be in the form of either a barplot, matrix plot, heatmap, or a dendrogram. I have represented bar graph. This shows existing values.

In [ ]:
msno.bar(df,figsize=(12,7))

- From this graph we can see that the null values in description variable is almost negligible but when we see CustomerID variable we can see there are almost 25% null values present.

#### Finding the number of unique customerID's

In [ ]:
print("Number of unique customers IDs:", len(df['CustomerID'].unique().tolist()))

For purposes of customer segmentation missing ids will be dropped.

### Descriptive statistics of quantitative variables

The describe() method returns description of the data in the DataFrame. If the DataFrame contains numerical data, the description contains these information for each column: count - The number of not-empty values. mean- The average (mean) value. This includes count, mean, median (or 50th percentile) standard variation. min-max, and percentile values of columns.

Descriptive statistics include those that summarize the central tendency, dispersion and shape of a dataset's distribution, excluding NaN values.

Analyzes both numeric and object series, as well as DataFrame column sets of mixed data types. The output will vary depending on what is provided.

In [ ]:
df.describe()

From descriptive statistics of quantitative variables (Quantity and UnitPrice), we could see that there are negative values in these variables, which are not possible values for price and quantity

## Data Cleaning and Univariate Analysis

### 2.1 Analysing the InvoiceNo variable

Let's have a closer look on the reason of negative values in these variables. In the description of dataset it is written that there are some canceled transactions presented in dataset. Canceled transactions is indicated by invoice number which starts with letter 'c'. Probably transactions with negative values of quantity are canceled.

Filtering transaction with invoice number started with letter 'C':

In [ ]:
len(df['InvoiceNo'].unique().tolist())

In [ ]:
cancelled = df[df['InvoiceNo'].astype(str).str.contains('C')]
cancelled.head()

Among cancelled transactions ther are not any transaction with positive quantity

In [ ]:
cancelled[cancelled['Quantity']>0]


However some transactions with negative quantity doesn't belong to cancelled:

In [ ]:
print("Number of cancelled transactions:", len(cancelled))
print("Total number of transaction with negative quantity:", df[df['Quantity'] <= 0 ]['Quantity'].count())

There are also transactions with zero price. Transactions with zero price doesn't bring usefull information to analysis, moreover mainly there are not any description for such transactions. This transactions could be deleted.

In [ ]:
df[df['UnitPrice'] < 0].head()

As it was mentioned before there are some negative quantity transactions wich are not cancelled, hovewer all these transactions have 0 price.

In [ ]:
d = df[~df['InvoiceNo'].astype(str).str.contains('C')]  
print("Number of of transactions with negative quantity value among non cancelled transactions:" ,len(d[d['Quantity']<=0]))
print("Number of of transactions with zero price and negative quantity" ,len(d[(d['Quantity']<=0) & (d['UnitPrice'] == 0)]))

In [ ]:

valid = d[(d['Quantity']>0) & (d['UnitPrice'] > 0)]
cancelled = d[(d['Quantity']<=0) & (d['UnitPrice']<=0)]
ser = pd.Series([len(valid)/(len(valid)+len(cancelled)),len(cancelled)/(len(valid)+len(cancelled))], index=['valid transactions','cancelled transactions'])

In [ ]:
ser.plot(kind='pie', autopct='%.2f%%',startangle=90,figsize=(7, 7)).set(ylabel='Transactions')

- we can see that around 0.25% of the tanscations are cancelled which are having unitprice and quantity as negative values the rest 99.75% of the transcations are valid transactions.

Percentage of transactions which will be deleted:

In [ ]:
print("Percentage of cancelled transactions:" , round(len(cancelled)/len(df)*100, 2),"%")
print("Precentage of transactions with zero or negative price" ,round(len(df[df['UnitPrice']<=0])/len(df)*100, 2),"%")

In [ ]:
data = df[(df['UnitPrice'] > 0) & (df['Quantity']>0)]

All rows with missing descriptions are gone after deleting problematic transactions.

In [ ]:
data.info()

### Check and delete duplicate transactions:

In [ ]:
print("Number of duplicated transactions:", len(data[data.duplicated()]))

In [ ]:
data.drop_duplicates(inplace = True)

### 2.2 CustomerID

### Dropping the rows having missing values

Drop rows with missing customers IDs:

I have used dropna() for customerID column sinces there are null values. The dropna() method removes the rows that contains NULL values. The dropna() method returns a new DataFrame object unless the inplace parameter is set to True, in that case the dropna() method does the removing in the original DataFrame instead

In [ ]:
 data = data.dropna(subset=['CustomerID'])
    

In [ ]:
data.shape

In [ ]:
top25cust =data.CustomerID.value_counts().index[:25]
top25cust

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_title('top 25 customers based on number of transactions made', fontsize=20)
sns.countplot(y='CustomerID', data=data, order=top25cust)
ax.set_xlabel('No.of Transactions', fontsize=15)
ax.set_ylabel('CustomerID', fontsize=15)
plt.show()

- these are rows left after removing the rows having null values, now our customerID column is cleaned.

In [ ]:
data.isnull().sum()

- now we can see that there are no null values present in the dataset, hence we proceed to find if any irregularities still exist in other columns

### 2.3 Descriptions

It should be taken into account that the same description could be written in different cases, let's convert all description into lowercase.

In [ ]:
data['Description'] = data['Description'].str.lower()

In [ ]:
print("Number of unique descriptions:", len(data['Description'].unique().tolist()))

- we can see that there are 3877 unique descriptions in this dataset which means this company sells these many variety of things.

In [ ]:
data.head()

#### Removing punctuations for the description:

In [ ]:
data['Description'] = data['Description'].replace(to_replace=r'[^\w\s]', value='', regex=True)

After this there are less unqiue descriptions:

In [ ]:
print("Number of unique descriptions:", len(data['Description'].unique().tolist()))

### 2.4 Stock Codes

Stock Code should be numerical value, hovewer there are some transactions which contains only multiple letters/words, this transactions will also be dropped

In [ ]:
data = data.loc[data['StockCode'].astype(str).str[0:4].str.isdigit()]

In [ ]:
data['StockCode'].unique()

In [ ]:
len(data['StockCode'].unique().tolist())

there are 3659 unique stockcodes are present

In [ ]:
top15stocks =data.StockCode.value_counts().index[:15]
top15stocks

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_title('top 15 stockcodes based on number of transactions made', fontsize=20)
sns.countplot(y='StockCode', data=data, order=top15stocks,palette= 'Set1')
ax.set_xlabel('No.of Transactions', fontsize=15)
ax.set_ylabel('Stockcodes', fontsize=15)
plt.show()

### 2.5 InvoiceDates

Adding features as year, month, month_year, day of week, hour:

In [ ]:
data['month'] = data['InvoiceDate'].dt.month
data['year'] = data['InvoiceDate'].dt.year

In [ ]:
data['WeekDay'] = data['InvoiceDate'].dt.day_name()

In [ ]:
data['month_year'] = pd.to_datetime(data[['year', 'month']].assign(Day=1))

In [ ]:
data['hour'] = data['InvoiceDate'].dt.hour

In [ ]:
data.head()

- Here you can see new columns are added to the table which makes our analysis easier 

In [ ]:
#which year have more orders
plt.figure(figsize=(8,5))
sns.countplot(data['year'],palette= 'Set1')
plt.xticks(rotation=40,ha='right')
plt.title("Year Distribution")
plt.xlabel('Year')
plt.ylabel('Count');


From this plot we can conclude that this dataset contains more transactions form 2011.

#### How many orders per month?

There are more purchasing made at the end of the year. The highest number of transactions were made during November. Probably reason could be the fact, that people tend to prepare for Christmas during November. Other than November number of byuings during the year is quite similar.

In [ ]:
plot4 = pd.DataFrame(data.groupby(['month_year'])['InvoiceNo'].count()).reset_index()
ax = sns.lineplot(x="month_year", y="InvoiceNo", data = plot4)

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(data['month'],palette= 'Set1')
plt.xticks(rotation=40,ha='right')
plt.title("Year Distribution")
plt.xlabel('Year')
plt.ylabel('Count');

Here we can see the october,november and december have the highest sales than the remaining months.

#### How many orders per day?

Interesting fact is that there were not any transaction on Saturday during the whole period. One of the explanations colud be some filter during the data collection process. Number of buyings are not significantly differ according to the day of the week.

In [ ]:
plt.figure(figsize=(8,5))
sns.countplot(data['WeekDay'],palette= 'Set1')
plt.xticks(rotation=40,ha='right')
plt.title("weekly Distribution")
plt.xlabel('weekday')
plt.ylabel('Count');

During the Wednesday and the Thursday there are high amount of sales that can sold so servers might be busy on those couple of days, are need to maintained correctly.

#### How many orders per hour?

The most number of transactions is done between 12 a.m. and 2 p.m., people tend to make there purchase during the lunch time. There aren't any transactions after 8 p.m. till 6 a.m.

In [ ]:
plot = pd.DataFrame(data.groupby(['hour'])['InvoiceNo'].count()).reset_index()
ax = sns.barplot(x="hour", y="InvoiceNo", data = plot)

### 2.6 UnitPrice

From the descriptive statistics for price variable, we could see that most ot the sold products are quite cheap. Probably retail sells some small items, as office supplies, small decorations etc.

In [ ]:
pd.DataFrame(data['UnitPrice'].describe())

In [ ]:
data[data['UnitPrice']>50]['Description'].unique().tolist()

Description of products, which cost more than 50 (Assumption: same currency for retails in different countries.)

### Price distribution of expensive goods :

In [ ]:
sns.distplot(data[data['UnitPrice']>50]['UnitPrice'], kde=True, rug=True);

### Removing Outliers

I have used plotly for better user experience and visualization.

I have removed outliers by using Box Plot. Box plots are used to show distributions of numeric data values. especially when you want to compare them between multiple groups. They are built to provide high-level information at a glance, offering general information about a group of data's symmetry, skew, variance, and outliers.

In [ ]:
px.box(data['UnitPrice'])

For removing Outilers, I have used IQR method. We can use the IQR method of identifying outliers to set up a "fence" outside of Q1 and Q3. Any values that fall outside of this fence are considered outliers. To build this fence we take 1.5 times the IQR and then subtract this value from Q1 and add this value to Q3.

In [ ]:
Q1 = np.percentile (data["UnitPrice"], 25, interpolation="midpoint")

Q3= np.percentile (data[ "UnitPrice"], 75, interpolation="midpoint")

IQR = Q3-Q1

min1 = Q1 - 1.5* IQR

max1 = Q3 + 1.5 *IQR

print(min1, max1, IQR, Q1, Q3)

In [ ]:
data = data[(data["UnitPrice"]> min1) & (data["UnitPrice"] < max1)]

data.shape

In [ ]:
px.box(data['UnitPrice'])

- Here we can see that maximum of the products are having having the unitprice ranging from 0 to 6.

### 2.7 Quantity

In [ ]:
pd.DataFrame(data['Quantity'].describe())

#### Identifing the outliers

I have removed outliers by using Box Plot. Box plots are used to show distributions of numeric data values. especially when you want to compare them between multiple groups. They are built to provide high-level information at a glance, offering general information about a group of data's symmetry, skew, variance, and outliers.

In [ ]:
px.box(data['Quantity'])

- we can see from above plot that there are outliers present in the Quantity column we need to handle these outliers by removing them from the dataframe.

For removing Outilers, I have used IQR method. We can use the IQR method of identifying outliers to set up a "fence" outside of Q1 and Q3. Any values that fall outside of this fence are considered outliers. To build this fence we take 1.5 times the IQR and then subtract this value from Q1 and add this value to Q3.

In [ ]:
Q1 = np.percentile (data["Quantity"], 25, interpolation="midpoint")

Q3= np.percentile (data[ "Quantity"], 75, interpolation="midpoint")

IQR = Q3-Q1

min1 = Q1 - 1.5* IQR

max1 = Q3 + 1.5 *IQR

print(min, max1, IQR, Q1, Q3)

In [ ]:
data = data[(data["Quantity"]> min1) & (data["Quantity"] < max1)]
data.shape

In [ ]:
px.box(data['Quantity'])

- Here we can see that most of the transactions have quantities in the range of 0 to 25.

In [ ]:
plt.figure(figsize=(10,5))
sns.distplot(data['Quantity'],bins=10, hist=True, kde=True)
plt.show()

- From the above dist plot we can see that most transactions have the quantity between 1 and 4.

### 2.8 Country

In [ ]:
data.Country.unique()

### 1. Based on your understanding of the data, what kind of business is this company in?


Answer - As the name suggest this company is more into online retail across 37 countries like

'United Kingdom', 'France', 'Australia', 'Netherlands', 'Germany', 'Norway', 'EIRE', 'Switzerland', 'Spain', 'Poland', 'Portugal', 'Italy', 'Belgium', 'Lithuania', 'Japan', 'Iceland', 'Channel Islands', 'Denmark', 'Cyprus', 'Sweden', 'Austria', 'Israel', 'Finland', 'Bahrain', 'Greece', 'Hong Kong', 'Singapore', 'Lebanon', 'United Arab Emirates', 'Saudi Arabia', 'Czech Republic', 'Canada', 'Unspecified', 'Brazil', 'USA', 'European Community', 'Malta', 'RSA'

In [ ]:
top10Countries = data.Country.value_counts().index[:10]
top10Countries

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_title('top 10 customers based on number of transactions made', fontsize=20)
sns.countplot(y='Country', data=data, order= top10Countries,palette="Set1")
ax.set_xlabel('No.of Transactions', fontsize=15)
ax.set_ylabel('Country', fontsize=15)
plt.show()

- Here in this above plot we can conclude that most of the orders are from United Kingdom followed by Germany
- And we can also that all the top 10 countries are belong to Europe Continent.

### Transactions by countries

The most unique customers are from United Kingdom, for this reason it is better to concetrate on UK for customer segmentation

In [ ]:
customer_country=data[['Country','CustomerID']].drop_duplicates()
customer_country.groupby(['Country'])['CustomerID'].aggregate('count').reset_index().sort_values('CustomerID', ascending=False)

In [ ]:
print("Number of transactions where country is unspecified:", len(data[data['Country']=='Unspecified']))

#### Here we can see that most of the transcations are from United Kingdom, now let's look at transactions from other countries

In [ ]:
no_uk=data[data.Country != 'United Kingdom']

In [ ]:
top_10=no_uk['Country'].value_counts().head(10)

In [ ]:
fig = plt.figure(figsize=(10,15))
sns.barplot(x =top_10.index,y=top_10.values,palette='Set1')
#'costumers from each country excluding UK'

- From this bar plot we can see that ohter that UK most orders comes from Germany and France.
- This is because these 2 countries are near to UK as comapred to other Europian Countries.

## Bivariate Analysis

### what is Bivariate Analysis?

Bivariate analysis is an analysis of two variables to determine the relationships between them. They are often reported in quality of life research. It is one of the simplest forms of quantitative (statistical) analysis. It involves the analysis of two variables (it is often denoted as X, Y), for the purpose of determining the empirical relationship between them.

### Q 2). Analyze the sales performance of this company, and provide your insights regarding the same

### Answer :

#### Performance Analysis
Sales Performance can be seen with

1) Number of sales every month<br>
2) Number of sales every year<br>
3) We see that in 2010 we have sales only for dec and in 2011 we have sales for all months<br>
4) We can see that from September to Novembor we have very good sales<br>
5) We could see that DEC 2010 we have sales of 748957 and in DEC 2011 we have sales of 433668 which is a huge drop when analyzed    further found out that We have only data upto 9th on dec 2011, so we find a sales drop in the month of dec 2011

### 3. Based on your analysis of the data, what are potential areas of improvement for the business?

In [ ]:
last5=data['Country'].value_counts().tail(5)
last5

In [ ]:
fig = plt.figure(figsize=(10,5))
sns.barplot(x =last5.index,y=last5.values,palette='Set2')

### Answer-
1) We see that september to december we have very high sales<br>
2) We can concentrate on improving the sales for the other 8 months<br>
3) We find very less number of customers in Lithania, Brazil, Czech Republic, Bahrain, Saudi Arabia<br>
4) We have very less sales for Lebanon, Brazil, RSA, Bahrain, Saudi Arabia. We can concentrate on improving their sales<br>
5) We could improve the business by increasing the customers and sales point of view

#### creating a new column "Revenue" for analysing the sales

In [ ]:
data['Revenue'] = data['UnitPrice'] * data['Quantity']

In [ ]:
data.head()

A new column is added named "Revenue" as you can see from the above table

In [ ]:
top15Revenues=data["Revenue"].value_counts().index[:15]
top15Revenues

In [ ]:
fig, ax = plt.subplots(figsize=(12, 8))
ax.set_title('top 15 Revenues based on number of transactions made', fontsize=20)
sns.countplot(y='Revenue', data=data, order=top15Revenues,palette= 'Set2')
ax.set_xlabel('No.of Transactions', fontsize=15)
ax.set_ylabel('Revenue', fontsize=15)
plt.show()

In [ ]:
# Top15 spend Most Client

Rev_Coustomer=data.groupby('CustomerID')['Revenue'].sum().sort_values(ascending=False).reset_index()[:15]
#Visualize
plt.figure(figsize=(12,5))
sns.barplot(x='CustomerID', y='Revenue', data=Rev_Coustomer)
plt.title("Top15 spend Most Customers ")
plt.xlabel('CustomerID')
plt.ylabel('Total Revenue')

In [ ]:
# Top20 Most Revenue Type
# Get grouped of total Revenue and Type
Rev_Country=data.groupby('Description')['Revenue'].sum().sort_values(ascending=False).reset_index()[:20]
#Visualize
plt.figure(figsize=(12,5))
sns.barplot(x='Description', y='Revenue', data=Rev_Country)
plt.xticks(rotation=40,ha='right')
plt.title("Total Revenue of Type")
plt.xlabel('Description')
plt.ylabel('Total Revenue')

- Here we can see that "Party Bunting" and "White Hanging heart t-Light Holder" these 2 items have been the top selling items for this company

### which month brings the highest Revenue?

In [ ]:
plot2 = pd.DataFrame(data.groupby(['month_year'])['Revenue'].sum()).reset_index()
ax = sns.lineplot(x = 'month_year', y='Revenue', data = plot2)

- The highest sales are also during the November, as the number of purchasing is the highest.This can because of festivals like Christmas and New year Eve.

In [ ]:
# Top20 Most Revenue Type
# Get grouped of total Revenue and Type
Rev_Country=data.groupby('StockCode')['Revenue'].sum().sort_values(ascending=False).reset_index()[:20]
#Visualize
plt.figure(figsize=(12,5))
sns.barplot(x='StockCode', y='Revenue', data=Rev_Country)
plt.xticks(rotation=40,ha='right')
plt.title("Total Revenue of Type")
plt.xlabel('Description')
plt.ylabel('Total Revenue')

- From the above plot we can see that products StockCode "47566" and "85123A" are the top selling products.

In [ ]:
#filtering customers by the total finalprice
Top10Customers = data.groupby(['Country'])['Revenue'].sum().sort_values(ascending=False)[0:10]
#creating the barplot
plt.figure(figsize=(8,5))
sns.barplot(x=Top10Customers.values, y=Top10Customers.index,palette= 'Set1').set(xlabel='Revenue',ylabel='Country')
plt.suptitle('Top10 Customers and Country of Origin by Sales Value', fontsize = 15)
plt.show()

Another statistic that we could use for the future planning, is how many of our customers are repeat customers, meaning that they bought products from us more than once. In the plot below we can see that almost 70% of the customers are repeat customers. In the other plot we can also see which customers from which countries had the most repeats.

In [ ]:
#grouping customers by the number of their visits and separating them
MostRepeat = data.groupby(['Country'])['InvoiceNo'].nunique().sort_values(ascending=False)
rep = MostRepeat[MostRepeat != 1].values
nrep = MostRepeat[MostRepeat == 1].values
ser = pd.Series([len(rep)/(len(rep)+len(nrep)),len(nrep)/(len(rep)+len(nrep))], index=['Repeat Customers','One-time Customers'])
#creating the subplot
fig, [ax1, ax2] = plt.subplots(nrows=1, ncols=2, figsize= (15,5), gridspec_kw= {'width_ratios':[1,1]})
plt.subplots_adjust(wspace=0.2)
sns.barplot(x=MostRepeat[0:10].values, y=MostRepeat[0:10].index, ax=ax1,palette= 'rocket').set(xlabel='Number of Transactions(Repeats)',ylabel='Country')
ser.plot(kind='pie', autopct='%.2f%%', ax=ax2).set(ylabel='')
plt.suptitle('Top Repeat Customers', fontsize=15)
plt.show()

### The most sales makes United Kingdom

In [ ]:
plot6 = pd.DataFrame(data.groupby(['Country'])['Revenue'].sum()).reset_index()
plot6 = plot6.sort_values(['Revenue']).reset_index(drop=True)
fig, ax = plt.subplots()
fig.set_size_inches(13, 11.5)
ax=sns.barplot(x='Country', y='Revenue',data=plot6.tail(10),estimator=max,ax=ax, palette='Set1')
ax.set_xticklabels(ax.get_xticklabels(), rotation=47, ha="right")
plt.show()

In [ ]:
plt.figure(figsize=(12,7))
sns.scatterplot (x=data['Country'], y=data['Revenue'])

plt.xlabel('Country')

plt.ylabel('Revenue')

plt.title('revenue in each country');

- From this Scatter plot we can see that almost in all the countries most orders have revenue below 60.

### 4. What are additional business problems that can be analyzed using this data. Support with explanation.

In [ ]:
# Sales Average of each product

avg_sales = data.groupby(['StockCode','Description'])['Quantity','Revenue'].mean().sort_values(by = 'Quantity',ascending = False)
avg_sales

### Answer -

1) We can see the demand for each product<br>
2) We can further check month wise on which product is on more demand for which month. If there is any seasonality for the         product<br>
3) Further deeper analysis can be carried out customer wise

## Multivariate Analysis

### What is “multivariate”?

Multivariate data analysis is a set of statistical models that examine patterns in multidimensional data by considering, at once, several data variables. It is an expansion of bivariate data analysis, which considers only two variables in its models. As multivariate models consider more variables, they can examine more complex phenomena and find data patterns that more accurately represent the real world.

In [ ]:
data.head(10)

In [ ]:
data.corr()

In [ ]:
plt.figure(figsize=(8,8))

plt.title('Correlation')

numeric_correlation=data.corr()
sns.heatmap(numeric_correlation, annot=True, square=True)

In [ ]:
print(numeric_correlation[ 'Revenue'].sort_values (ascending=False))

- Here we can see that only Quantity is somewhat related to revenue,and there is not much relation from other attributes.

In [ ]:
nonuk=data[data.Country != 'United Kingdom']
fig = px.line(nonuk, x="InvoiceDate", y="Revenue", color="month",title="Sales by Country in 2011", hover_name="Country",
        line_shape="spline", render_mode="svg")
fig.show()

- here 90% of the data is having the country as "UK" ,I have ploted this graph excluding the UK so that we can analyse the sales of remaining countries 
- From this line plot we can see that the highest sales is in the month of november and december.
- and in the month august and september the sales is low 

### Group by and get multiple statistics for different columns

Calculating receny, frequency, and the customer lifetime value

In [ ]:
#Getting recency frequency and customer lifetime value
RFCLTV = data.groupby(
   ['Country', 'CustomerID'], as_index=False
).agg(
    {
         'InvoiceNo':'count',    # count number of orders for each customer in each country
         'Revenue': "sum",  # get the count of networks
         'InvoiceDate': 'last'  # get the last date per group
    }
)
RFCLTV.head()

### Combining date based aggregations and group by multiple column aggregations

naming grouped aggregate columns with multiple statistics, reset index to include the date column

In [ ]:
agg_data = data.groupby([pd.Grouper(key='InvoiceDate', freq='MS'), 'Country'])\
    .agg(unique_items=('StockCode', 'nunique'),
         total_quantity=('Quantity','sum'),
         total_amount=('Revenue','sum')).reset_index()
agg_data_uk = agg_data[agg_data['Country']=='United Kingdom']

In [ ]:
agg_data_uk.head()

In [ ]:
fig = px.box(nonuk, x="Country", y="Revenue")
fig.show()

- here in the above box plot we can see that there some outliers 
- apart form we can see that the more revenue is generated from countries that belong to Europe that others.

In [ ]:
res = pd.pivot_table(data=nonuk,index="WeekDay",columns="Country",values="Quantity")
res

In [ ]:
#Scatter plot
plt.figure(figsize=(6,3))
fig = px.scatter(RFCLTV, x="InvoiceNo", y="Revenue", title = 'Frequency vs Customer Lifetime Value')
fig.show()

- Here in above scatter plot we can see that mostly the invoice numbers range between 0 to 2000.
- and only few orders crossed the 200k mark.
- most of the sales are having the revenue in the rage of 0 to 50k. 

In [ ]:
fig = px.histogram(data, x="month_year" , nbins = 30)
fig.update_layout(bargap=0.2)
fig.show()

## 3. Analysis

### 3.1 Top Customers

Calculate how often did particular customer make purchase during the year:

In [ ]:
data['freq'] = data.groupby('CustomerID')['CustomerID'].transform('count')

Calculate the top 10 customers who brings the highest revenue

In [ ]:
customer  = pd.DataFrame(data.groupby(['CustomerID'])['Revenue'].sum().sort_values(ascending=False)).reset_index().head(10)

In [ ]:
customer['CustomerID'].tolist()

In [ ]:
top_customer = data[data['CustomerID'].isin(customer['CustomerID'].tolist())][['CustomerID', 'Country', 'Revenue', 'Quantity', 'freq']]
top_customer.head()

## 4.Conclusions


-	We have analysed the complete data and cleaned it by dealing with the irregularites which may hamper the further processes.
-	We have analsed variable by variable to understand the patterns and find if any anomalies left. 
-	After analysing the variables we found that:
-	The data contains the transactions from 2010 dec to 2011 dec.
-	There are certain cancelled transactions we are handled.
-	The company is having Saturday as Holiday.
-	During the months of November and December there is a huge sale, so better to keep more stock during these months.
-	There are certain stocks which are in more demand.
-	Most of the customers are from UK lie around 90%, apart from them second largest country in terms of orders is “Germany”.
-	There are some worst performing countries like Lebanon, Brazil, RSA, Bahrain, Saudi Arabia. We can concentrate on improving their sales.
-	We could improve the business by increasing the customers and sales point of view
-	We can also see that items having less unit price are sold more than items having high unitprice.

#####   From Bivariate and multivariate analysis we can see that

-	we can see that "Party Bunting" and "White Hanging heart t-Light Holder" these 2 items have been the top selling items for this company’
-	The highest sales are also during the November, as the number of purchasing is the highest.This can because of festivals like Christmas and New year Eve.
-	  we can see that products StockCode "47566" and "85123A" are the top selling products.
- Another statistic that we could use for the future planning, is how many of our customers are repeat customers, meaning that they bought products from us more than once. In the plot we can see that almost 70% of the customers are repeat customers. In the other plot we can also see which customers from which countries had the most repeats.
 
-	From this Scatter plot we can see that almost in all the countries most orders have revenue below 60.
-	From Heatmap we can see that only Quantity is somewhat related to revenue,and there is not much relation from other attributes.
-	From this line plot we can see that the highest sales is in the month of november and december.
-	and in the month august and september the sales is low.


### 5. Recommendations

There are problems in data collections and data integration processes which should be solved, for more effective data analysis and more clear sollutions.

1. Missing customer ID - this column could be made not nullable, in case for treatment occurences of missing IDs.<br>
2. Not valid Stock Codes (contains different letters/some words) - some checking constraints for stock codes.<br>
3. Duplicate transasctions - probably data were collected from multiple sources, which caused some duplictities.<br>
4. To each stock code should belong only one product description, this rule is not kept in dataset. The reason could be again absence of constraints or wrong joins of the tables, if the dataset were constricted from multiple tables.<br>
5. Solving of problems related to data quality could significantly improve the results of algorithms.

Based on RFM analysis several customers segments from UK could be defined. There are quite big precantage of loyal customers or customers who tend to spned big amount of money while buying. Also ther groups of customers who are already lost and who are going to be lost in near future. Uqing this groups some targeting campaignes could be used.